In [2]:
# Functions: Analyse the evoltion of average positive and negative
import numpy as np
import pandas as pd
import dask.dataframe as dd
from scipy import sparse
import time
from fun.fun import *

c:\Users\stirl\miniconda3\envs\network_env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [42]:
# -> IN : Read Edge List
edges_fn = "../data/edges.parquet"
print("reading edges ... ", end='')
start = time.time()
df = dd.read_parquet("../data/edges.parquet")
df[['source', 'target']] = df[['source', 'target']].astype('int32')
end = time.time()
print("read {:_} lines (took {:.1f}s)".format(len(df), (end-start)))
df

reading edges ... read 684_732_453 lines (took 0.0s)
source    int32
target    int32
dtype: object


,source,target
npartitions=157,,
,int32,int32
,...,...
...,...,...
,...,...
,...,...


In [15]:
# extract unique nodes
nodes = dd.concat([df['source'], df['target']]).unique().compute()
num_nodes = len(nodes)
print("found {:_} unique nodes".format(num_nodes))

found 32 unique nodes


In [37]:
# csr
node_index = {node: index for index, node in enumerate(nodes)}
print(df.head())
mp = pd.DataFrame()
mp['source'] = df['source'].apply(lambda x: node_index[x])
mp['target'] = df['target'].apply(lambda x: node_index[x])

row = mp['source']
col = mp['target']
data = np.ones_like(row)
adj_mat = sparse.csr_matrix((data, (row, col)), shape=(num_nodes, num_nodes), dtype=bool)
#adj_mat.toarray()

   source  target
0      13  103151
1      13  214293
2  103151  214293
3      13  138731
4      13   42023


In [40]:
import dask.array as da

x = da.ones(15, chunks=3)
x

dask.array<ones_like, shape=(15,), dtype=float64, chunksize=(3,), chunktype=numpy.ndarray>

In [6]:
# 
n = 400_000
entries = int(n*n*0.004)
print("{:_}".format(entries))
row = np.random.randint(0, n-1, size=(entries), dtype='int32')
col = np.random.randint(0, n-1, size=(entries), dtype='int32')
data = np.ones_like(col)

64_000_000


In [ ]:
# create csr matrix
#mat = sparse.csr_matrix((data, (row, col)), shape=(n,n), dtype='int8')
mat = sparse.csr_matrix((data, (row, col)), shape=(n,n), dtype=bool)
show_n = 10
print(mat[0:show_n,0:35].toarray())

In [46]:
# save to npz
mat_fn = '../data/sparse_test.npz'
sparse.save_npz(mat_fn, mat)

In [ ]:
# load from npz
#mat = sparse.load_npz(mat_fn)
#print(mat[0:20,0:35].toarray())
print(mat.toarray())

In [ ]:
# Create a mapping from node IDs to indices
node_to_index = {node: index for index, node in enumerate(nodes)}

# Function to process a partition of the DataFrame and build adjacency matrix
def build_adjacency_matrix(partition):
    adjacency = np.zeros((len(nodes), len(nodes)), dtype=np.uint8)
    for index, row in partition.iterrows():
        source_index = node_to_index[row['source']]
        target_index = node_to_index[row['target']]
        adjacency[source_index, target_index] = 1
        adjacency[target_index, source_index] = 1  # If graph is undirected
    return adjacency

# Build adjacency matrix in chunks
chunk_size = 10_000
adjacency_chunks = [
    sparse.csr_matrix(build_adjacency_matrix(partition))
    for partition in df.to_delayed()
]

# Concatenate chunks to get the final adjacency matrix
adjacency_matrix = sum(adjacency_chunks)

# Save adjacency matrix to a file
np.savez_compressed('adjacency_matrix.npz', data=adjacency_matrix.data, indices=adjacency_matrix.indices, indptr=adjacency_matrix.indptr, shape=adjacency_matrix.shape)


In [10]:
s = 2
mat = np.zeros((s,s), dtype='int8')
print(mat)
print(type(mat[0,0]))

[[0 0]
 [0 0]]
<class 'numpy.int8'>


In [ ]:
# 
adj_matrix = np.zeros((num_nodes, num_nodes), dtype='int8')
processed, total = 0, len(df)
for i, j in df.values:
    adj_matrix[i, j] = 1
    adj_matrix[j, i] = 1
    processed, _ = track_progress(total, processed, text='rows processed:', inc=50)
print("Done")